In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

# laoding London Tube data from tubedata.csv into a DataFrame df.
df = pd.read_csv('tubedata.csv', header=None)
df.head()

# processes the df to extract info about stations, lines, zones, and associated costs 
# and stored in dictionaries station_dict and zone_dict.

station_dict = defaultdict(list)
zone_dict = defaultdict(set)
# get data row by row
for index, row in df.iterrows():
  
  start_station = row[0]
  end_station = row[1]
  act_cost = int(row[3])

  zone1 = row[4]
  zone2 = row[5]
  # Extracting line information - added this change to support in UCS cost function
  line = row[2]  
    
  # station dictionary of child station tuples (child_name, cost from parent to the child)
  # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
  station_list = station_dict[start_station]
  station_list.append((end_station, act_cost, line))

  # the following two lines add the other direction of the tube "step"
  station_list = station_dict[end_station]
  station_list.append((start_station, act_cost,line))

 # we add the main zone
  zone_dict[start_station].add(zone1)
  # we add the secondary zone
  if zone2 != "0":
    zone_dict[start_station].add(zone2)
    # if the secondary zone is not 0 it's the main zone for the ending station
    zone_dict[end_station].add(zone2)
  else:
    # otherwise the main zone for the ending station is the same as for the starting station
    zone_dict[end_station].add(zone1)

# Since some zones are given as a,b,c,d and in the actual tube map its is 7,8,9,10 
#Hence mapping from alphabet zones to numerical equivalents so that heuristic distance calculation is made easy
zone_to_numeric = {'a': 7, 'b': 8, 'c': 9, 'd': 10}

# Function to replace alphabet zones with integers
def replace_alphabet_zones(zone_set):
    return {zone_to_numeric[zone] if zone.isalpha() else zone for zone in zone_set}

# Apply the replacement to the entire zone_dict
zone_dict = {station: replace_alphabet_zones(zones) for station, zones in zone_dict.items()}




#This function takes a node and constructs the path from the root (initial station) to that node in the graph.
#It returns the path and the total cost associated with the path.

def construct_path_from_root(node):
    # Initialize the path with the label of the input node
    path_from_root = [node['label']]
    # Set the current_node to the input node
    current_node = node
    # Initialize the total_cost variable to keep track of the total cost of the path
    total_cost = 0
    
    # While loop to traverse the path back to the root
    while current_node['parent']:
        # Calculate the cost in terms of the average time between the current node and its parent using find_cost function
        cost = find_cost(station_dict, current_node['parent']['label'], current_node['label'])
        # Update the total_cost by adding the calculated cost
        total_cost += cost
        # Move one step up the path by updating the current_node to its parent
        current_node = current_node['parent']
        # Update the path_from_root list by adding the label of the current node at the beginning
        path_from_root = [current_node['label']] + path_from_root
    
    # Return the constructed path from root and the total cost of the path
    return path_from_root, total_cost


# function to find the cost in terms of the average time of traveling from a parent station to a child station
def find_cost(station_dict, parent, child_label):
    # iterate through each of station_dict[parent] 
    for child, cost, line in station_dict[parent]:
        # if the child station matches the specified child_label, return its cost
        if child == child_label:
            return cost
            
    return None

# Function to perform depth-first graph search on a station_dict graph
def my_depth_first_graph_search(station_dict, initial, goal, compute_exploration_cost=True, reverse=False):
    # Initialize the frontier with the initial node
    frontier = [{'label': initial, 'parent': None, 'line': None}]
    # Initialize the set of explored nodes with the initial node
    explored = {initial}
    # Initialize the count of explored nodes
    number_of_explored_nodes = 0

    # While frontier has nodes
    while frontier:
        # pop from the right of the list
        node = frontier.pop()
        # increment number_of_explored_nodes
        number_of_explored_nodes += 1
        
        # Check if the current node is the goal
        if node['label'] == goal:
            # if compute_exploration_cost is true, print the number of explored nodes
            if compute_exploration_cost:
                print('DFS:Number of nodes  expanded = {}'.format(number_of_explored_nodes))
            # Return the constructed path from root to the goal node
            return construct_path_from_root(node)

        # Get the neighbors of the current node from the station_dict
        neighbors = station_dict[node['label']]
        
        # if the reverse flag is True, reverse the order of neighbors
        if reverse:
            neighbors = reversed(neighbors)

        # iterate through each neighbor
        for child_label, cost, line in neighbors:
            # child node
            child = {'label': child_label, 'parent': node, 'line': None}
            # if the child is not in the set of explored nodes
            if child_label not in explored:
                frontier.append(child) # added to the right of the list, so it is a LIFO
                # add the child to the set of explored nodes
                explored.add(child_label)
    
    return None


# Function to perform breadth-first graph search on a station_dict graph
def my_breadth_first_graph_search(station_dict, initial, goal, compute_exploration_cost=True, reverse=False):

    frontier = [{'label': initial, 'parent': None, 'line': None}]
    explored = {initial}
    number_of_explored_nodes = 0

    while frontier:
        node = frontier.pop()  # pop from the right of the list
        number_of_explored_nodes += 1
        if node['label'] == goal:
            if compute_exploration_cost:
                print('BFS:Number of nodes expanded = {}'.format(number_of_explored_nodes))
            # Return the constructed path from root to the goal node
            return construct_path_from_root(node)
            
        # Get the neighbors of the current node from the station_dict
        neighbors = station_dict[node['label']]

         # if the reverse flag is True, reverse the order of neighbors
        if reverse:
            neighbors = reversed(neighbors)
            
        # iterate through each neighbor
        for child_label, cost, line in neighbors:
            child = {'label': child_label, 'parent': node, 'line': None}
            if child_label not in explored:
                frontier = [child] + frontier # added to the left of the list, so a FIFO!
                explored.add(child_label)
    return None

      
from queue import PriorityQueue

# Function to perform my_uniform_cost_search search on a station_dict graph
#Perform Uniform Cost Search to find the shortest path from starting_station to destination_station
def my_uniform_cost_search(station_dict, initial_station, goal_station, compute_exploration_cost=True, reverse=False):

    if initial_station == goal_station:  # Just in case, because now we are checking the children
        return None
        
    # Instantiate a PriorityQueue
    # PriorityQueue always select the path with the lowest cost/priority first.
    
    frontier = PriorityQueue()
    # Insert the initial node with priority 0
    frontier.put((0, (initial_station, initial_station, 0)))  

    number_of_explored_nodes = 0
    explored = {initial_station}

    # Explore nodes until the frontier is empty
    while not frontier.empty():
        # Get the node with the lowest priority from the priority queue
        _, (current_station, path, cost) = frontier.get()
        number_of_explored_nodes += 1

        # Check if the current station is the destination
        if current_station == goal_station:
            print("UCS Number of nodes expanded:", number_of_explored_nodes)
            print("UCS Path in stations:", path)
            print("UCS Total cost in average time:", cost)
            return True

        # Get neighbors of the current station
        neighbors = station_dict[current_station]
        # reverse if false is true
        if reverse:
            neighbors = reversed(neighbors)

        # iterate through neighbors
        for neighbor, chid_cost, line in neighbors:
            # if the neighbor has not been explored
            if neighbor not in explored:
                # update the path and cost for the neighbor
                new_path = path + " -> " + neighbor
                # upadte new cost is the sum of cost till parent , the cost to neighbor node
                new_cost = cost + chid_cost
                # add the neighbor to the priority queue with new_cost, and new_path
                frontier.put((new_cost, (neighbor, new_path, new_cost)))
                # mark the neighbor as explored
                explored.add(neighbor)

    return False


# Testing for Euston to victoria
initial_station = "Euston" 
goal_station = "Victoria" 

'''
# Testing for Canada Water to Stratford
initial_station = "Canada Water" 
goal_station = "Stratford" 

# Testing for New Cross Gate to Stepney Green
initial_station = "New Cross Gate" 
goal_station = "Stepney Green" 

# Testing for Ealing Broadway to South Kensington
initial_station = "Ealing Broadway" 
goal_station = "South Kensington" 


# Testing for Baker Street to Wembley Park
initial_station = "Baker Street" 
goal_station = "Wembley Park" 

'''
solution = my_depth_first_graph_search(station_dict, initial_station, goal_station, True, False)
print("DFS:Path in stations:", solution[0])
print("DFS:Total cost in average time:", solution[1])

solution = my_breadth_first_graph_search(station_dict, initial_station, goal_station, True, False)
print("BFS:Path in stations:", solution[0])
print("BFS:Total cost in average time:", solution[1])

solution = my_uniform_cost_search(station_dict, initial_station, goal_station, True, False)



DFS:Number of nodes  expanded = 25
DFS:Path in stations: ['Euston', "King's Cross St. Pancras", 'Russell Square', 'Holborn', 'Covent Garden', 'Leicester Square', 'Piccadilly Circus', 'Green Park', 'Victoria']
DFS:Total cost in average time: 13
BFS:Number of nodes expanded = 35
BFS:Path in stations: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
BFS:Total cost in average time: 7
UCS Number of nodes expanded: 35
UCS Path in stations: Euston -> Warren Street -> Oxford Circus -> Green Park -> Victoria
UCS Total cost in average time: 7


In [2]:
from queue import PriorityQueue

#Extended Uniform Cost Search to find the shortest path from starting_station to destination_station.
#uses the current UCS cost function to include the time to change lines at one station here uses 2 minutes.
def my_uniform_cost_search_extended(station_dict, starting_station, destination_station, compute_exploration_cost=True, reverse=False):

    # Initialize a priority queue with the initial node
    frontier = PriorityQueue()
    frontier.put((0, (starting_station, starting_station, 0, None))) # Initial node with priority 0 and no current line

    number_of_explored_nodes = 0
    explored = {starting_station}

    # Explore nodes until the frontier is empty
    while not frontier.empty():
        # Get the node with the lowest priority from the priority queue
        _, (current_station, path, cost, current_line) = frontier.get()
        number_of_explored_nodes += 1

        # Check if the current station is the destination
        if current_station == destination_station:
            print("Extended UCS: Number of nodes expanded:", number_of_explored_nodes)
            print("Extended UCS: Path in stations:", path)
            print("Extended UCS: Total cost in average time:", cost)
            return True

        # Get neighbors of the current station
        neighbors = station_dict[current_station]
        if reverse:
            neighbors = reversed(neighbors)

        # Explore neighbors
        for neighbor, step_cost, line in neighbors:
            # Calculate the cost of changing lines
            neighbor_line = line
            #if current_line and neighbor_line are not the same means there is a line change
            line_change_cost = 2 if current_line is not None and current_line != neighbor_line else 0

            # Check if the neighbor has not been explored
            if neighbor not in explored:
                # Update the path, cost, and line for the neighbor
                new_path = path + " -> " + neighbor
                #same as previous UCS in case of line change line_change_cost is added
                new_cost = cost + step_cost + line_change_cost
                frontier.put((new_cost, (neighbor, new_path, new_cost, neighbor_line)))
                # Mark the neighbor as explored
                explored.add(neighbor)

    return False

solution = my_uniform_cost_search_extended(station_dict, initial_station, goal_station, True, False)


Extended UCS: Number of nodes expanded: 35
Extended UCS: Path in stations: Euston -> Warren Street -> Oxford Circus -> Green Park -> Victoria
Extended UCS: Total cost in average time: 9


2.4 Heuristic

In [3]:
# Testing my_best_first_search

initial_station = "Baker Street" 
goal_station = "Wembley Park" 
initial_station = "Ealing Broadway" 
goal_station = "South Kensington" 
def heuristic_distance(l1, l2):
    common_zones = set(l1) & set(l2)

    if common_zones:
        # Case 1: Two lists have common zones, consider that and return 0
        return 0
    else:
        # Case 2: No common zones, compare sorted lists
        sorted_l1 = sorted(map(int, l1))
        sorted_l2 = sorted(map(int, l2))

        if sorted_l1[-1] > sorted_l2[0]:#if zones in l1 is greater than zones in l2
            # Last element of sorted l1 is greater than the first element of sorted l2
            min_l1 = min(sorted_l1)
            max_l2 = max(sorted_l2)
            return abs(min_l1 - max_l2)
        else:
            # Case 3: Last element of sorted l1 is less than or equal to the first element of sorted l2
            max_l1 = max(sorted_l1)
            min_l2 = min(sorted_l2)
            return abs(max_l1-min_l2)
#l2=[3,1,2]
#l1=[6,7]
#print(heuristic_distance(l1,l2))
def my_best_first_search(station_dict,starting_station, destination_station,compute_exploration_cost=False):
    
    queue = PriorityQueue()
    queue.put((0, (starting_station, starting_station, 0)))
    number_of_explored_nodes = 0
    explored = {starting_station}
    while not queue.empty():
        _, (current_station, path, cost) = queue.get()
        number_of_explored_nodes += 1
        if current_station == destination_station:
            print("Best-First Search  Path found:", path)
            print("Best-First Search  Cost:", cost)
            print("Best-First Search Number of nodes expanded:", number_of_explored_nodes)
            return True
        
        neighbors = station_dict[current_station]
        #if reverse:
            #neighbors = reversed(neighbors)

        for neighbor, step_cost, line in neighbors:
            if neighbor not in explored:
                new_path = path + " -> " + neighbor
                new_cost = cost + step_cost
                l1=list(zone_dict[neighbor])
                l2=list(zone_dict[destination_station])
                #heuristic_value=abs(int(a)-int(b))
                heuristic_value = heuristic_distance(l1, l2)
                #print("l1 {} l2 {} and h value {}".format(l1,l2,abs(heuristic_value)))
                queue.put((heuristic_value, (neighbor, new_path, new_cost)))
                explored.add(neighbor)

    return False

solution = my_best_first_search(station_dict, initial_station, goal_station,True)

#Zone-based heuristic which assigns a constant time of 10 minutes for travel within one zone and
#a constant time of 20 minutes for travel across zones.
def heuristic_distance_modified(l1, l2):
    common_zones = set(l1) & set(l2)

    if common_zones:
        return 10
    else:
        return 20


Best-First Search  Path found: Ealing Broadway -> Ealing Common -> Acton Town -> Turnham Green -> Hammersmith -> Barons Court -> Earls' Court -> Gloucester Road -> South Kensington
Best-First Search  Cost: 20
Best-First Search Number of nodes expanded: 57


3.1 Implement a Genetic Algorithm

In [4]:
import string
import hashlib
import math
import random
import copy
import numpy as np

# Function to generate the true password
def get_password(student_username, l=10):
    options = string.digits + string.ascii_uppercase + "_"
    h = hashlib.sha256(("ECS759P-AI" + student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
        s += options[n % len(options)]
    return s[0:l]

# True password for the given EECS username
student_password = get_password('eecs23740')#ec23746
print("True Password:", student_password)

# Genetic Algorithm to find the password
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score

MAX_VALUE = distance_function('0000000000', '__________')

def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE

    return ordered_dict

True Password: I7PE8X374T


In [5]:
# function to create initial population with given pop_size
#uses generate_random_password function
def create_init_population(pop_size):
    return [generate_random_password() for _ in range(pop_size)]
# generates random passwords with 10 characters long and it can contain only upper-case English letters,
#numbers between 0 and 9 (inclusive), and the underscore symbol
def generate_random_password(length=10):
    options = string.digits + string.ascii_uppercase + "_"
    return ''.join(random.choice(options) for _ in range(length))

In [6]:
# Sort the population based on fitness scores in descending order
#this is to get best individuals in the first of the population
def sort_individuals(population, student_password):
    # Calculate fitness scores for the population
    fitness_scores = get_normalised_fitness(population, student_password)

    # Sort the population based on fitness scores in descending order if reverse=True
    sorted_population = sorted(population, key=lambda x: fitness_scores[x], reverse=True)

    return sorted_population, fitness_scores

In [7]:
def crossing_over(pool_to_cross, pCO=0.1):
    num_reproductions = 0  # to get the number of num_reproductions in each cross over
    for index, indiv in enumerate(pool_to_cross):
        #print("cross over : index {} and indiv {}".format(i,indiv))
        # Randomly decide whether to perform crossover or not
        #If crossing over should be applied on this individual for the given crossover rate
        if random.random() <= pCO:
            num_reproductions += 1  # If cross over is performed , increment num_reproductions
            # Create a copy of the pool to cross (others) and remove the current individual
            # We get the list of all the other selected individuals
            others = pool_to_cross[:]
            others.remove(indiv)

            # Randomly choose another individual from the pool
            other = others[np.random.choice(len(others))]
            otherIdx = pool_to_cross.index(other)

            # Randomly choose a starting point for crossover
            startingIdx = np.random.choice(len(indiv))
            # Perform crossover
            ''''
            tempChange = indiv[startingIdx:]
            indiv[startingIdx:] = other[startingIdx:]
            other[startingIdx:] = tempChange
            '''
            #modified the above methods as indiv is a string here 
            tempChange = indiv[startingIdx:]
            indiv = indiv[:startingIdx] + other[startingIdx:]
            other = other[:startingIdx] + tempChange
            # Update the pool
            pool_to_cross[otherIdx] = other
            pool_to_cross[index] = indiv

    return pool_to_cross,num_reproductions

In [8]:
import numpy as np

def mutation(pool_to_mutate, pMuta=0.5):
    num_reproductions = 0  # # to get the number of num_reproductions in each mutation
    for index, indiv in enumerate(pool_to_mutate):
        for pos in range(len(indiv)):
            # Check whether to perform mutation or not
            if np.random.rand() <= pMuta:
                num_reproductions += 1  # If mutation is performed , increment num_reproductions
                # Randomly choose another position for mutation
                pos2 = random.sample(range(len(indiv)), 1)[0]

                # Swap the positions
                indiv_list = list(indiv)
                # Swap the characters at pos and pos2
                indiv_list[pos], indiv_list[pos2] = indiv_list[pos2], indiv_list[pos]
    
                # Convert the list back to a string
                indiv = ''.join(indiv_list)
                #print(indiv)
                '''
                temp = indiv[pos]
                indiv[pos] = indiv[pos2]
                indiv[pos2] = temp
                '''
    return pool_to_mutate,num_reproductions

In [9]:

# Population size
pop_size = 300
half = int(pop_size / 2)
# Mutation rate (per gene)
pMuta = 0.1
# Crossover rate
pCO = 0.8
# Number of generations
n_gen = 400

# Create initial population with random passwords
population = create_init_population(pop_size)
#print("Initial population generated by random password: {}".format(population))

 # Sort the population according to fitness, so best parents will be present in first part of population
population, fitness_scores = sort_individuals(population, student_password)

#find the total_reproductions in each generation by cross over and mutation
total_reproductions = 0

for i in range(n_gen):
    mating_pool = copy.deepcopy(population)[:half]
    
    # Perform crossover and mutation with specified rates
    mating_pool, num_reproductions_crossover = crossing_over(mating_pool, pCO)
    total_reproductions += num_reproductions_crossover
    
    mating_pool, num_reproductions_mutation = mutation(mating_pool, pMuta)
    total_reproductions += num_reproductions_mutation

    # Combine the best of parents and offspring to form a new population
    population = copy.deepcopy(population)[:half] + mating_pool
    #print("New population after combining the best of parents and offspring {}".format(population))

    # Sort population according to fitness
    population, fitness_scores = sort_individuals(population, student_password)
    
print("Total number of reproductions:", total_reproductions)
population, fitness_scores = sort_individuals(population, student_password)
print("Final results n_gen {} pop_size {} generations".format(n_gen,pop_size))
for individual in population:
    password = individual
    score = fitness_scores[individual]
    #print("Password:", password, "Fitness Score:", score)
# Get the best suitable password (the one with the highest fitness score)
best_password = population[0]
best_fitness_score = fitness_scores[best_password]
print("True Password:", student_password)
print("Best Suitable Password:", best_password)
print("Fitness Score:", best_fitness_score)

Total number of reproductions: 108265
Final results n_gen 400 pop_size 300 generations
True Password: I7PE8X374T
Best Suitable Password: I7PE8X374T
Fitness Score: 1.0


In [10]:

# Number of runs
num_runs = 10

# Store the number of reproductions for each run
reproductions_per_run = []

for run in range(num_runs):
    # Population size
    pop_size = 300
    half = int(pop_size / 2)
    # Mutation rate (per gene)
    pMuta = 0.1
    # Crossover rate
    pCO = 0.8
    # Number of generations
    n_gen = 400

    # Create initial population with random passwords
    population = create_init_population(pop_size)

    # Sort the population according to fitness, so best parents will be present in first part of population
    population, fitness_scores = sort_individuals(population, student_password)
    
    # Initialize the counter
    total_reproductions = 0

    for i in range(n_gen):
        mating_pool = copy.deepcopy(population)[:half]

        # Perform crossover and mutation with specified rates
        mating_pool, num_reproductions_crossover = crossing_over(mating_pool, pCO)
        total_reproductions += num_reproductions_crossover

        mating_pool, num_reproductions_mutation = mutation(mating_pool, pMuta)
        total_reproductions += num_reproductions_mutation

        # Combine the best of parents and offspring to form a new population
        population = copy.deepcopy(population)[:half] + mating_pool

        # Sort population according to fitness
        population, fitness_scores = sort_individuals(population, student_password)

    # Store the number of reproductions for this run
    reproductions_per_run.append(total_reproductions)
    # Sort the population according to fitness
    population, fitness_scores = sort_individuals(population, student_password)
    print("Run {}: Total number of reproductions: {}".format(run + 1, total_reproductions))

# Calculate mean and standard deviation
mean_reproductions = np.mean(reproductions_per_run)
std_dev_reproductions = np.std(reproductions_per_run)

print("\nMean number of reproductions over {} runs: {}".format(num_runs, mean_reproductions))
print("Standard deviation of reproductions over {} runs: {}".format(num_runs, std_dev_reproductions))
#population, fitness_scores = sort_individuals(population, student_password)
print("Final results n_gen {} pop_size {} generations".format(n_gen,pop_size))
for individual in population:
    password = individual
    score = fitness_scores[individual]
    #print("Password:", password, "Fitness Score:", score)
# Get the best suitable password (the one with the highest fitness score)
best_password = population[0]
best_fitness_score = fitness_scores[best_password]
print("True Password:", student_password)
print("Best Suitable Password:", best_password)
print("Fitness Score:", best_fitness_score)

Run 1: Total number of reproductions: 108411
Run 2: Total number of reproductions: 108136
Run 3: Total number of reproductions: 107828
Run 4: Total number of reproductions: 108218
Run 5: Total number of reproductions: 108216
Run 6: Total number of reproductions: 107705
Run 7: Total number of reproductions: 107894
Run 8: Total number of reproductions: 108015
Run 9: Total number of reproductions: 107823
Run 10: Total number of reproductions: 107598

Mean number of reproductions over 10 runs: 107984.4
Standard deviation of reproductions over 10 runs: 244.98293818141704
Final results n_gen 400 pop_size 300 generations
True Password: I7PE8X374T
Best Suitable Password: I7PE8X374T
Fitness Score: 1.0


In [11]:
# Define ranges for the variables
#tried combos 
'''
pMuta_values = [0.1, 0.2, 0.3]
pCO_values = [0.7, 0.8, 0.9]
pop_size_values = [100, 200, 300]
n_gen_values = [400, 500, 600]
'''
pMuta_values = [0.1, 0.2]
pCO_values = [0.7,0.8, 0.9]
pop_size_values = [300]
n_gen_values = [400]
# Iterate over all combinations
for pMuta in pMuta_values:
    for pCO in pCO_values:
        for pop_size in pop_size_values:
            for n_gen in n_gen_values:
                # Create initial population with random passwords
                population = create_init_population(pop_size)

                # Sort the population according to fitness, so best parents will be present in first part of population
                population, fitness_scores = sort_individuals(population, student_password)

                for i in range(n_gen):
                    mating_pool = copy.deepcopy(population)[:half]
                    #print("Gen {} initial best first half of population: mating_pool {}".format(i, mating_pool))

                    # Perform crossover and mutation with specified rates
                    mating_pool,_ = crossing_over(mating_pool, pCO)
                    mating_pool,_ = mutation(mating_pool, pMuta)

                    #print("After crossover and mutation mating_pool {}".format(mating_pool))

                    # Combine the best of parents and offspring to form a new population
                    population = copy.deepcopy(population)[:half] + mating_pool
                    #print("New population after combining the best of parents and offspring {}".format(population))

                    # Sort population according to fitness
                    population, fitness_scores = sort_individuals(population, student_password)

                # Print the final results for each combination
                #print("Results - pMuta: {}, pCO: {}, pop_size: {}, n_gen: {}".format(pMuta, pCO, pop_size, n_gen))
                # Sort the population according to fitness
                population, fitness_scores = sort_individuals(population, student_password)

                # Get the best suitable password (the one with the highest fitness score)
                #population, fitness_scores = sort_individuals(population, student_password)
                best_password = population[0]
                best_fitness_score = fitness_scores[best_password]
                if(best_fitness_score==1):
                    print("Results - pMuta: {}, pCO: {}, pop_size: {}, n_gen: {}".format(pMuta, pCO, pop_size, n_gen))
                    print("True Password:", student_password)
                    print("Best Suitable Password:", best_password)
                    print("Fitness Score:", best_fitness_score)
                    print("\n" + "="*40 + "\n")

Results - pMuta: 0.1, pCO: 0.8, pop_size: 300, n_gen: 400
True Password: I7PE8X374T
Best Suitable Password: I7PE8X374T
Fitness Score: 1.0


Results - pMuta: 0.1, pCO: 0.9, pop_size: 300, n_gen: 400
True Password: I7PE8X374T
Best Suitable Password: I7PE8X374T
Fitness Score: 1.0


Results - pMuta: 0.2, pCO: 0.8, pop_size: 300, n_gen: 400
True Password: I7PE8X374T
Best Suitable Password: I7PE8X374T
Fitness Score: 1.0


Results - pMuta: 0.2, pCO: 0.9, pop_size: 300, n_gen: 400
True Password: I7PE8X374T
Best Suitable Password: I7PE8X374T
Fitness Score: 1.0


